# Vaccine Temp pipeline objectives
Notes:

Report ID = [38,39] [2137, 2111]

A:
1) export report: "Export facility info for temperature logs upload", #38 (2137)
2) add field: "redcap_event_name" = "facility_arm_1", for all records
3) add field: "facility_info_complete" = 2, for all records
4) set field "redcap_event_name" as second index
5) import into PID278

B:
1) export report: "Export storage info for temperature logs upload", #39 (2111)
2) add field: "redcap_event_name" = "month_year_arm_1", month spelled out, all lower case, 4 digit integer for year
3) add field: "storage_info_complete" = 2, for all records
4) rename columns according to:

{
 "waiis_fac_name":"waiis_fac_name_s",
 "loc_vfc_pin":"loc_vfc_pin_s",
 "org_name":"org_name_s"
 }

5) set field "redcap_event_name" as second index
6) import into PID278


In [1]:
import redcap


In [2]:
%run -i -n "credentials.py"

In [3]:
print(dev_url)
print(prod_url)

print(["dev_pid100", "dev_pid278", "prod_pid100", "prod_pid278"])

https://qa-redcap.doh.wa.gov/api/
https://redcap.doh.wa.gov/api/
['dev_pid100', 'dev_pid278', 'prod_pid100', 'prod_pid278']


In [4]:
prod100 = redcap.Project(prod_url, prod_pid100)

### Experiment with "export_survey_fields" feature

In [8]:
df_prod100 = prod100.export_records(format_type = "df")

C:\Users\AXG5303\Miniconda3\envs\transform22\lib\site-packages\redcap\methods\base.py:137: DtypeWarning: Columns (397) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(buf, **df_kwargs)


In [5]:
#Report ID = [38,39] [2137, 2111]
report2137_df = prod100.export_report("2137" ,format_type = "df")
report2111_df = prod100.export_report("2111", format_type = "df")

In [9]:
df_prod100.shape

(2670, 471)

In [7]:
df_prod100_v2 = prod100.export_records(format_type = "df", export_survey_fields = True)

C:\Users\AXG5303\Miniconda3\envs\transform22\lib\site-packages\redcap\methods\base.py:137: DtypeWarning: Columns (1,397,407,422) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(buf, **df_kwargs)


In [ ]:
df_prod100_v2['redcap_survey_identifier']

In [9]:
df_prod100_v2.shape

(2663, 483)

In [10]:
set(df_prod100_v2.columns) - set(df_prod100.columns)

{'agreement_and_signatures_timestamp',
 'agreement_prescreen_timestamp',
 'facility_information_timestamp',
 'facility_storage_timestamp',
 'inquiry_enrollment_survey_timestamp',
 'isa_approval_timestamp',
 'list_of_providers_timestamp',
 'organization_information_timestamp',
 'prescreening_survey_timestamp',
 'redcap_survey_identifier',
 'vaccine_management_team_review_timestamp',
 'waiis_team_review_timestamp'}

In [11]:
df_prod100_v2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2663 entries, 1 to 8203
Columns: 483 entries, redcap_survey_identifier to vaccine_management_team_review_complete
dtypes: float64(101), int64(68), object(314)
memory usage: 9.8+ MB


In [12]:
df_prod100_v2['redcap_survey_identifier']

record_id
1                   Kindred Hospital Seattle - First Hill
2                             East Adams Rural Healthcare
3         Virginia Mason Health System - Corporate Office
4                                 Overlake Medical Center
5                          Ferry County Memorial Hospital
6               Suquamish Tribe Community Health Programs
7                                   North Valley Hospital
8                                                     NaN
9                                                     NaN
10             Providence Regional Medical Center Everett
11                           Seattle Cancer Care Alliance
12                           Kaiser Permanente Washington
13                                  Coulee Medical Center
14                                  Three Rivers Hospital
15                                Virginia Mason Memorial
16                       Providence Mount Carmel Hospital
17      Providence Sacred Heart Medical Center & Child...
19  

In [13]:
meta = prod100.export_metadata(format_type = "df")

In [14]:
A = prod100.export_records(format_type = "df", forms = ['inquiry_enrollment_survey'])

In [15]:
B = prod100.export_records(format_type = "df", forms = [ 'inquiry_enrollment_survey','inquiry_review_form', 'prescreening_survey'])

## EXPORT PID278 - Explore metadata such as forms and events 

In [16]:
prod278 = redcap.Project(prod_url, prod_pid278)

In [52]:
prod278_df = prod278.export_records(format_type = "df", record_type = "flat")

In [116]:
report2137_df.columns.shape

(7,)

In [124]:
report2137_df.columns

Index(['vm_approved', 'org_name', 'waiis_fac_name', 'org_email',
       'loc_vtrcks_id', 'loc_vfc_pin', 'loc_type'],
      dtype='object')

In [125]:
report_clms = list(report2137_df.columns) +  list(report2111_df.columns)
report_clms.remove("vm_approved")

In [126]:
pd.Series(report_clms).value_counts()

org_name                      1
storage_type2                 1
loc_store_frozen_num          1
loc_store_ult_frozen          1
approval_status_ultracold     1
approval_status_freezer       1
approval_status               1
pfizer_storage                1
storage_type1                 1
storage_model1                1
storage_manu1                 1
storage_model2                1
waiis_fac_name                1
storage_manu2                 1
storage_type3                 1
storage_model3                1
storage_manu3                 1
storage_type4                 1
storage_manu4                 1
storage_model4                1
storage_type5                 1
storage_manu5                 1
loc_store_frozen              1
loc_store_ult_frozen_num      1
loc_store_refrigerated_num    1
loc_store_refrigerated        1
org_email                     1
loc_vtrcks_id                 1
loc_vfc_pin                   1
loc_type                      1
vm_approved                   1
loc_prim

In [128]:
len(report_clms)

46

In [130]:
overlap = set(report_clms) & set(prod278_df.columns)

In [132]:
len(overlap)

46

In [94]:
idx = pd.IndexSlice

In [ ]:
prod278_df.loc[idx[:,"facility_arm_1"], idx[:]]

In [177]:
stack_df = prod278_df[['org_name', 'waiis_fac_name', 'org_email', 'loc_vtrcks_id','loc_vfc_pin', 'loc_type', 'batch_group', 'facility_info_complete', 'org_name_s']].copy()
stack_df.drop("1-TEST", level = "record_id", inplace = True)

In [178]:
stack_df_unstack = stack_df.unstack()

In [182]:
stack_df_unstack

org_name  \
redcap_event_name                                     facility_arm_1   
record_id                                                              
1                                         Kindred Hospital - Seattle   
2                                  Adams County Hospital District #2   
3                                   Virginia Mason Franciscan Health   
4                                   Overlake Hospital Medical Center   
5                                              Ferry County Memorial   
7                               Okanogan County Hospital district #4   
8                                      Columbia County Health System   
9                       Mason General Hospital and Family of Clinics   
10                                      PROVIDENCE HEALTH & SERVICES   
11                 Fred Hutchinson CRC, Occupational Health (2506...   
12                       Kaiser Foundation Health Plan of Washington   
13                                             Coulee Medical Center   
14                             Okanogan Douglas District Hospital #1   
15                   Yakima Valley Memorial Hospital Employee Health   
16                                      PROVIDENCE HEALTH & SERVICES   
17                                      PROVIDENCE HEALTH & SERVICES   
19                                                       PeaceHealth   
20                                  Pacific County Health Department   
21                 Summit Pacific Medical Center (aka Mark Reed H...   
22                        Lincoln County Public Hospital District #3   
23                 Public Hospital District No. 1 of King County ...   
24                                              Jefferson Healthcare   
25                                      PROVIDENCE HEALTH & SERVICES   
26                                               Grant County PHD #3   
27                              Clallam County Public Hospital No. 1   
29                                      PROVIDENCE HEALTH & SERVICES   
32                                Chelan County Hospital District #2   
33                                           Swedish Health Services   
34                                          BHC Fairfax Hospital Inc   
35                                      PROVIDENCE HEALTH & SERVICES   
...                                                              ...   
7852                                          SKAGIT REGIONAL HEALTH   
7858                                                 City of Seattle   
7871                                         Ketcher & Edwards, PLLC   
7876                                           Whitman Medical Group   
7880                                                  VIRGINIA MASON   
7887                            Jefferson Healthcare Sheridan Clinic   
7907                           Jefferson Healthcare Watership Clinic   
7908                            Jefferson Healthcare Townsend Clinic   
7919                          Pediatric Associates of Whidbey Island   
7930                                       Northwoods Lodge Pharmacy   
7954                                                        JIKE LLC   
7960                                                Jogan Health LLC   
7966                                Consistent Care Support Services   
7969                         Iora Health Washington Physicians, P.C.   
7976                                                nguyen' pharmacy   
8011                             Public Health Seattle & King County   
8014                                           PSPD Marysville, PLLC   
8023               EVERGREEN HEALTHCARE - DBA: KC HOSP DIST #2 (1...   
8024               EVERGREEN HEALTHCARE - DBA: KC HOSP DIST #2 (1...   
8025               EVERGREEN HEALTHCARE - DBA: KC HOSP DIST #2 (1...   
8026               EVERGREEN HEALTHCARE - DBA: KC HOSP DIST #2 (1...   
8043               EVERGREEN HEALTHCARE - DBA: KC HOSP DIST #2 (1...   
8048                        

In [181]:
stack_df_unstack.melt?

Signature:
stack_df_unstack.melt(
    id_vars=None,
    value_vars=None,
    var_name=None,
    value_name='value',
    col_level: 'Level | None' = None,
    ignore_index: 'bool' = True,
) -> 'DataFrame'
Docstring:
Unpivot a DataFrame from wide to long format, optionally leaving identifiers set.

This function is useful to massage a DataFrame into a format where one
or more columns are identifier variables (`id_vars`), while all other
columns, considered measured variables (`value_vars`), are "unpivoted" to
the row axis, leaving just two non-identifier columns, 'variable' and
'value'.

Parameters
----------
id_vars : tuple, list, or ndarray, optional
    Column(s) to use as identifier variables.
value_vars : tuple, list, or ndarray, optional
    Column(s) to unpivot. If not specified, uses all columns that
    are not set as `id_vars`.
var_name : scalar
    Name to use for the 'variable' column. If None it uses
    ``frame.columns.name`` or 'variable'.
value_name : scalar, default 'val

In [179]:
stack_df

org_name  \
record_id redcap_event_name                                                      
1         facility_arm_1                            Kindred Hospital - Seattle   
          march_2022_arm_1                                                 NaN   
2         facility_arm_1                     Adams County Hospital District #2   
          march_2022_arm_1                                                 NaN   
3         facility_arm_1                      Virginia Mason Franciscan Health   
          march_2022_arm_1                                                 NaN   
4         facility_arm_1                      Overlake Hospital Medical Center   
          march_2022_arm_1                                                 NaN   
5         facility_arm_1                                 Ferry County Memorial   
          march_2022_arm_1                                                 NaN   
7         facility_arm_1                  Okanogan County Hospital district #4   
          march_2022_arm_1                                                 NaN   
8         facility_arm_1                         Columbia County Health System   
          march_2022_arm_1                                                 NaN   
9         facility_arm_1          Mason General Hospital and Family of Clinics   
          march_2022_arm_1                                                 NaN   
10        facility_arm_1                          PROVIDENCE HEALTH & SERVICES   
          march_2022_arm_1                                                 NaN   
11        facility_arm_1     Fred Hutchinson CRC, Occupational Health (2506...   
          march_2022_arm_1                                                 NaN   
12        facility_arm_1           Kaiser Foundation Health Plan of Washington   
          march_2022_arm_1                                                 NaN   
13        facility_arm_1                                 Coulee Medical Center   
          march_2022_arm_1                                                 NaN   
14        facility_arm_1                 Okanogan Douglas District Hospital #1   
          march_2022_arm_1                                                 NaN   
15        facility_arm_1       Yakima Valley Memorial Hospital Employee Health   
          march_2022_arm_1                                                 NaN   
16        facility_arm_1                          PROVIDENCE HEALTH & SERVICES   
          march_2022_arm_1                                                 NaN   
...                                                                        ...   
8011      facility_arm_1                   Public Health Seattle & King County   
          march_2022_arm_1                                                 NaN   
8014      facility_arm_1                                 PSPD Marysville, PLLC   
          march_2022_arm_1                                                 NaN   
8023      facility_arm_1     EVERGREEN HEALTHCARE - DBA: KC HOSP DIST #2 (1...   
          march_2022_arm_1                                                 NaN   
8024      facility_arm_1     EVERGREEN HEALTHCARE - DBA: KC HOSP DIST #2 (1...   
          march_2022_arm_1                                                 NaN   
8025      facility_arm_1     EVERGREEN HEALTHCARE - DBA: KC HOSP DIST #2 (1...   
          march_2022_arm_1                                                 NaN   
8026      facility_arm_1     EVERGREEN HEALTHCARE - DBA: KC HOSP DIST #2 (1...   
          march_2022_arm_1                                                 NaN   
8043      facility_arm_1     EVERGREEN HEALTHCARE - DBA: KC HOSP DIST #2 (1...   
          march_2022_arm_1                                                 NaN   
8048      facility_arm_1                              Jamestown Healing Clinic   
          march_2022_arm_1                                                 NaN   
8049      facility_arm_1                            

In [174]:
stack_df.melt(id_vars = "redcap_event_name", value_vars = "org_name")

,redcap_event_name,variable,value
0,facility_arm_1,org_name,Kindred Hospital - Seattle
1,march_2022_arm_1,org_name,NaN
2,facility_arm_1,org_name,Adams County Hospital District #2
3,march_2022_arm_1,org_name,NaN
4,facility_arm_1,org_name,Virginia Mason Franciscan Health
5,march_2022_arm_1,org_name,NaN
6,facility_arm_1,org_name,Overlake Hospital Medical Center
7,march_2022_arm_1,org_name,NaN
8,facility_arm_1,org_name,Ferry County Memorial
9,march_2022_arm_1,org_name,NaN


In [66]:
prod278_df.loc["1"]

,pause___1,org_name,waiis_fac_name,org_email,loc_vtrcks_id,loc_vfc_pin,loc_type,batch_group,facility_info_complete,org_name_s,...,followup_unit_5,approved_followup_5,notes_unit_5,update,other_info_v2203,monthly_upload_complete,date_reviewed,reviewer,reviewer_notes,review_complete
redcap_event_name,,,,,,,,,,,,,,,,,,,,,
facility_arm_1,0.0,Kindred Hospital - Seattle,KINDRED HOSPTIAL-SEATTLE,Shannon.Stone@kindred.com,WAAP81021,P81021,9.0,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
march_2022_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kindred Hospital - Seattle,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0


In [ ]:
prod278_df.loc

In [ ]:
prod278.export_records?

In [ ]:
prod278_df = prod278.export_project_info

In [17]:
meta278_df = prod278.export_metadata(format_type = "df")

In [ ]:
meta278_df

In [19]:
prod278.is_longitudinal

True

In [ ]:
prod100.is_longitudinal

In [20]:
instrument_event_map = prod278.export_instrument_event_mappings(format_type = "df")

In [21]:
instrument_event_map

,arm_num,unique_event_name,form
0,1,facility_arm_1,facility_info
1,1,march_2022_arm_1,storage_info
2,1,march_2022_arm_1,monthly_upload
3,1,march_2022_arm_1,review
4,1,april_2022_arm_1,storage_info
5,1,april_2022_arm_1,monthly_upload
6,1,april_2022_arm_1,review
7,1,may_2022_arm_1,storage_info
8,1,may_2022_arm_1,monthly_upload
9,1,may_2022_arm_1,review


In [23]:
meta278_df.columns

Index(['form_name', 'section_header', 'field_type', 'field_label',
       'select_choices_or_calculations', 'field_note',
       'text_validation_type_or_show_slider_number', 'text_validation_min',
       'text_validation_max', 'identifier', 'branching_logic',
       'required_field', 'custom_alignment', 'question_number',
       'matrix_group_name', 'matrix_ranking', 'field_annotation'],
      dtype='object')